In [10]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [11]:
# Create network model
model = Network('model')

# Block 1: Create nodes
node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PSPRIO)  # Preemptive Static Priority

In [12]:
# Block 2: Create job classes with different priorities
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 2, node[0], 0)  # 2 jobs, priority 0 (low)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[0], 1)  # 2 jobs, priority 1 (high)

In [13]:
# Block 3: Set service time distributions - following MATLAB specification exactly
# Delay station (think time)
node[0].set_service(jobclass[0], Erlang(3, 2))  # Erlang(k=3, rate=2)
node[0].set_service(jobclass[1], HyperExp(0.5, 3.0, 10.0))  # HyperExp(p=0.5, rate1=3.0, rate2=10.0)

# PSPRIO Queue station
node[1].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))  # HyperExp(p=0.1, rate1=1.0, rate2=10.0)
node[1].set_service(jobclass[1], Exp(1))  # Exponential with rate=1

In [14]:
# Block 4: Set routing (serial routing for both classes) - following MATLAB specification exactly
P = model.init_routing_matrix()
# P{1} = Network.serialRouting(node) equivalent for jobclass[0]
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)  # Delay -> Queue1
P.set(jobclass[0], jobclass[0], node[1], node[0], 1.0)  # Queue1 -> Delay
# P{2} = Network.serialRouting(node) equivalent for jobclass[1]
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  # Delay -> Queue1
P.set(jobclass[1], jobclass[1], node[1], node[0], 1.0)  # Queue1 -> Delay
model.link(P)

In [15]:
# Model information
M = model.get_number_of_stations()
K = model.get_number_of_classes()

In [ ]:
# SolverCTMC removed - not tested in JAR for prio_psprio
# solver_ctmc = SolverCTMC(model)
# avg_table_ctmc = solver_ctmc.get_avg_table()

In [17]:
# Solve with JMT simulation - skip due to PSPRIO JMT support issue
print("\\nSOLVER: JMT (skipped - PSPRIO scheduling not supported in JMT)")
print("JMT cannot handle PSPRIO scheduling strategy - ClassNotFoundException for EPSStrategyPriority")
# solver_jmt = SolverJMT(model, seed=23000, verbose=VerboseLevel.STD, samples=5000)
# avg_table_jmt = solver_jmt.get_avg_table()

\nSOLVER: JMT (skipped - PSPRIO scheduling not supported in JMT)
JMT cannot handle PSPRIO scheduling strategy - ClassNotFoundException for EPSStrategyPriority


In [ ]:
# SolverSSA with seed=23000, samples=5000, verbose=SILENT to match JAR test
solver_ssa = SolverSSA(model, seed=23000, samples=5000, verbose=VerboseLevel.SILENT)
avg_table_ssa = solver_ssa.get_avg_table()